# Melanoma detection in dermatoscopic images using contextual information and Convolutional Neural Networks.
By Brenda Farinha Fernandes

November 2022

## Setup

Required libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import zipfile
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split 
from matplotlib import pyplot as plt

import tensorflow as tf

Mounted at /content/drive


GPU Information

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
  print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

strategy = tf.distribute.get_strategy()
REPLICAS = strategy.num_replicas_in_sync
print('Number of replicas:', REPLICAS)

Sun Feb 26 12:11:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    48W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

RAM Information

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


Global variables

In [4]:
PATH_dataset_images_zip = '/content/drive/MyDrive/PFG_MELANOMA/ISIC_2020_Training_JPEG.zip'
PATH_dataset_images = '/content/drive/MyDrive/PFG_MELANOMA/images'
PATH_dataset_csv = '/content/drive/MyDrive/PFG_MELANOMA/ISIC_2020_Training_GroundTruth.csv'

In [5]:
IMG_HEIGHT = 124
IMG_WIDTH = 124

In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64

## Data preproccesing

Read [SIIM-ISIC 2020 Melanoma Classification Challenge Dataset](https://doi.org/10.34970/2020-ds01) 

In [7]:
def read_dataset(dataset_unzip = False): 

  # Unzip images from "/content/drive/MyDrive/PFG_MELANOMA/ISIC_2020_Training_JPEG.zip"
  if dataset_unzip: 
    zipfile.ZipFile(PATH_dataset_images_zip).extractall(PATH_dataset_images)
  
  df = pd.DataFrame(pd.read_csv(f'{PATH_dataset_csv}'))

  train_df, val_df = train_test_split(df, random_state=42, shuffle=True, stratify=df['target'])

  return train_df, val_df

In [8]:
def decode_image(image_path, label):

  image = tf.io.read_file(image_path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
  image = tf.cast(image, tf.float32) / 255.0

  return image, label

In [9]:
def image_augmentation(image, label): 

  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_flip_up_down(image)
  image = tf.image.random_hue(image, 0.025)
  image = tf.image.random_saturation(image, 0.6, 1.4)
  image = tf.image.random_contrast(image, 0.7, 1.4)
  image = tf.image.random_brightness(image, 0.1)

  return image, label

Load dataset: 
* `map()`: This transformation applies map_func to each element of this dataset, and returns a new dataset containing the transformed elements

* `repeat()`: Repeats this dataset so each original value is seen count times.

* `shuffle()`: Randomly shuffles the elements of this dataset.

* `batch()`: Combines consecutive elements of this dataset into batches.

* `prefetch()` : Most dataset input pipelines should end with a call to prefetch. This allows later elements to be prepared while the current element is being processed. prefetch( ) doesn’t allow CPU stand idle. When model is training prefetch continue prepare data while GPU is busy.

* `cache()` : The first time the dataset is iterated over, its elements will be cached either in the specified file or in memory. Subsequent iterations will use the cached data. When caching to a file, the cached data will persist across runs. Even the first iteration through the data will read from the cache file.



In [10]:
def load_dataset(df, deterministic = False, augment = False, cache = False): 

  labels = df["target"].values
  images = f'{PATH_dataset_images}/' + df["image_name"].values +".jpg"
  
  ds = tf.data.Dataset.from_tensor_slices((images, labels))

  if not deterministic:
    options = tf.data.Options()
    options.experimental_deterministic = False
    ds = ds.with_options(options)

  ds = ds.map(decode_image, num_parallel_calls=AUTOTUNE)

  if augment: 
    ds = ds.map(image_augmentation, num_parallel_calls=AUTOTUNE)
    ds = ds.repeat()
    ds.shuffle(buffer_size = 2048)
  
  ds = ds.batch(BATCH_SIZE)

  if cache:
    ds = ds.cache()

  ds = ds.prefetch(AUTOTUNE)

  return ds

Show images

In [11]:
def show_images(ds):

  row, col = 3,5
  fig = plt.figure(figsize=(2*col, 2*row))
  for _row in range(row):
    for _col in range(col):
      plt.subplot(row, col, _row*col + _col +1)
      for images, labels in ds.take(1):
        img = images[0].numpy()
        plt.imshow(img)
        plt.title(labels[0].numpy())
        plt.axis(False)
  plt.show()

## Model

In [12]:
train_df, val_df = read_dataset(dataset_unzip = False)

Distribution dataset

In [13]:
train_total = len(train_df)
train_malignant = len(train_df[train_df["target"] == 1])
train_benign = len(train_df[train_df["target"] == 0])

val_total = len(val_df)
val_malignant = len(val_df[val_df["target"] == 1])
val_benign = len(val_df[val_df["target"] == 0])

print("Number of training files =", train_total)
print("\tNumber of malignant training files =", train_malignant)
print("\tNumber of benign training files =", train_benign)

print("\nNumber of validation files =", val_total)
print("\tNumber of malignant validation files =", val_malignant)
print("\tNumber of benign validation files =", val_benign)

Number of training files = 24843
	Number of malignant training files = 437
	Number of benign training files = 24406

Number of validation files = 8282
	Number of malignant validation files = 146
	Number of benign validation files = 8136


In [14]:
STEPS_PER_EPOCH_TRAIN = train_total // BATCH_SIZE
STEPS_PER_EPOCH_VAL = val_total // BATCH_SIZE

EPOCHS = 15
print("Number of steps per epoch in training:", STEPS_PER_EPOCH_TRAIN)
print("Number of steps per epoch in validation:", STEPS_PER_EPOCH_VAL)

Number of steps per epoch in training: 388
Number of steps per epoch in validation: 129


In [ ]:
train_ds = load_dataset(train_df, deterministic = False, augment = True, cache = False)
val_ds = load_dataset(val_df, deterministic = False, augment = False, cache = True)

Class weights in training dataset

In [16]:
weight_malignant = (train_total/train_malignant)/2.0
weight_benign = (train_total/train_benign)/2.0

class_weight = {0: weight_benign, 1: weight_malignant}

print("Weight for benign cases = ", class_weight[0])
print("Weight for malignant cases = ", class_weight[1])

Weight for benign cases =  0.5089527165451119
Weight for malignant cases =  28.424485125858123


Defining Callbacks

*   `EarlyStopping`: Stop training when a monitored metric has stopped improving.
*   `ModelCheckpoint`: Callback to save the Keras model or model weights at some frequency.



In [17]:
callback_early_stopping = tf.keras.callbacks.EarlyStopping(
    patience = 15, 
    verbose = 0, 
    restore_best_weights = True)

callbacks_lr_reduce = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = "val_auc", 
    factor = 0.1, 
    patience = 10,
    verbose = 0, 
    min_lr = 1e-6)

callback_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "melanoma_detection_weights.h5",
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True)

In [18]:
with strategy.scope():

  input = tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
  dummy = tf.keras.layers.Lambda(lambda x:x)(input)

  encoder = tf.keras.applications.EfficientNetB6(
      input_shape = (IMG_HEIGHT, IMG_WIDTH, 3),
      include_top = False,
      weights = 'imagenet'
  )(dummy)
  encoder.trainable = False

  x = tf.keras.layers.GlobalAveragePooling2D()(encoder)
  outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

  model = tf.keras.Model(input, outputs, name='aNetwork')
  model.summary()

  model.compile(
      optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
      loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = 0.05),
      metrics = [tf.keras.metrics.AUC(name='auc')]
  )

Model: "aNetwork"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 124, 124, 3)]     0         
                                                                 
 lambda (Lambda)             (None, 124, 124, 3)       0         
                                                                 
 efficientnetb6 (Functional)  (None, 4, 4, 2304)       40960143  
                                                                 
 global_average_pooling2d (G  (None, 2304)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 2305      
                                                                 
Total params: 40,962,448
Trainable params: 40,738,009
Non-trainable params: 224,439
________________________________________

In [19]:
history_B6 = model.fit(
    train_ds, 
    verbose=1, 
    steps_per_epoch=STEPS_PER_EPOCH_TRAIN, 
    validation_data = val_ds, validation_steps = STEPS_PER_EPOCH_VAL,
    epochs = EPOCHS, 
    callbacks=[callback_early_stopping, callbacks_lr_reduce, callback_checkpoint],
    class_weight = class_weight) 

Epoch 1/15
388/388 [==============================] - 317s 535ms/step - loss: 0.6477 - auc: 0.7232 - val_loss: 0.8439 - val_auc: 0.7611 - lr: 0.0010
Epoch 2/15
388/388 [==============================] - 204s 527ms/step - loss: 0.5810 - auc: 0.7833 - val_loss: 1.1211 - val_auc: 0.7157 - lr: 0.0010
Epoch 3/15
388/388 [==============================] - 214s 553ms/step - loss: 0.5258 - auc: 0.8318 - val_loss: 0.6775 - val_auc: 0.8325 - lr: 0.0010
Epoch 4/15
388/388 [==============================] - 230s 595ms/step - loss: 0.5043 - auc: 0.8478 - val_loss: 0.6339 - val_auc: 0.4135 - lr: 0.0010
Epoch 5/15
388/388 [==============================] - 214s 553ms/step - loss: 0.4947 - auc: 0.8536 - val_loss: 1.2062 - val_auc: 0.7465 - lr: 0.0010
Epoch 6/15
388/388 [==============================] - 225s 582ms/step - loss: 0.5253 - auc: 0.8312 - val_loss: 0.7873 - val_auc: 0.6260 - lr: 0.0010
Epoch 7/15
388/388 [==============================] - 247s 638ms/step - loss: 0.4977 - auc: 0.8520 - val_l

In [25]:
model.evaluate(val_ds)

130/130 [==============================] - 50s 386ms/step - loss: 0.4361 - auc: 0.7905


[0.4360809326171875, 0.7904637455940247]